# CCAT Test-G3

This notebook contains analysis of simulated G3 files sent to me by Sourav Sarkar. Analyzing these files is a useful practice excercise, and provides a look into how the G3 format is used within the CCAT collaboration.

The data used can be obtained at https://drive.google.com/file/d/1C_zmKswTygGWSmXbx7ChctNZ4ty6yfxn/view.

Total download size: ~4MB

In [103]:
from spt3g import core
import so3g
import numpy as np
import matplotlib.pyplot as plt
import sys

In [71]:
filename = "test_g3/test_patch-0-1/frames-0000.g3"

In [72]:
class PrintFrameType(core.G3Module):
    def __init__(self):
        super(PrintFrameType, self).__init__()
        self.previous_type = None
        self.num_repeats = 0
    def Process(self, frame):
        type = frame.type
        if type == self.previous_type:
            self.num_repeats += 1
            print(f"{type} (x{self.num_repeats + 1})", end='\r')
        else:
            print()
            print(f"{type}", end='\r')
            self.num_repeats = 0
        self.previous_type = type

pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=filename)
pipe.Add(PrintFrameType)

pipe.Run()


Observation
Calibration
PipelineInfo
Scan (x42)


In [73]:
obsframe = list(core.G3File(filename))[0]
print(obsframe)
for detector_set in obsframe["observation_detector_sets"]:
    print(detector_set)

Frame (Observation) [
"observation_detector_sets" (spt3g.core.G3VectorVectorString) => 54 elements
"observation_name" (spt3g.core.G3String) => "test_patch-0-1"
"observation_uid" (spt3g.core.G3Int) => 2893290703
"scan_el" (spt3g.core.G3Double) => 0.541925
"scan_el_astropy_units" (spt3g.core.G3String) => "deg"
"scan_el_units" (spt3g.core.G3Int) => 6
"scan_max_az" (spt3g.core.G3Double) => 2.25621
"scan_max_az_astropy_units" (spt3g.core.G3String) => "rad"
"scan_max_az_units" (spt3g.core.G3Int) => 6
"scan_max_el" (spt3g.core.G3Double) => 0.541925
"scan_max_el_astropy_units" (spt3g.core.G3String) => "rad"
"scan_max_el_units" (spt3g.core.G3Int) => 6
"scan_min_az" (spt3g.core.G3Double) => 2.15057
"scan_min_az_astropy_units" (spt3g.core.G3String) => "rad"
"scan_min_az_units" (spt3g.core.G3Int) => 6
"scan_min_el" (spt3g.core.G3Double) => 0.541925
"scan_min_el_astropy_units" (spt3g.core.G3String) => "rad"
"scan_min_el_units" (spt3g.core.G3Int) => 6
"session_class" (spt3g.core.G3String) => "toast.

In [74]:
calframe = list(core.G3File(filename))[1]
print(calframe)

Frame (Calibration) [
"el_noise_model" (spt3g.core.G3VectorUnsignedChar) => 26468 elements
"el_noise_model_class" (spt3g.core.G3String) => "toast.noise.Noise"
"focalplane" (spt3g.core.G3VectorUnsignedChar) => 13861 elements
"noise_model" (spt3g.core.G3VectorUnsignedChar) => 29578 elements
"noise_model_class" (spt3g.core.G3String) => "toast.noise_sim.AnalyticNoise"
]


In [57]:
pipeinfoframe = list(core.G3File(filename))[2]
print(pipeinfoframe)

Frame (PipelineInfo) [
"12-Mar-2024:13:52:01.477128000" (spt3g.core.G3PipelineInfo) => master branch, no local diffs
]


In [110]:
firstscanframe = list(core.G3File(filename))[3]
print(firstscanframe)
first_quat = firstscanframe["boresight_azel"][0]
print(so3g.proj.quat.decompose_lonlat(first_quat))
print(firstscanframe["azimuth"][0])
print(firstscanframe["elevation"][0])

Frame (Scan) [
"azimuth" (spt3g.core.G3VectorDouble) => 2541 elements
"boresight_azel" (spt3g.core.G3VectorQuat) => 2541 elements
"boresight_radec" (spt3g.core.G3VectorQuat) => 2541 elements
"compress_flags_D00A-280_gain" (spt3g.core.G3Int) => 1
"compress_flags_D00A-280_offset" (spt3g.core.G3Double) => 0
"compress_flags_D00A-280_units" (spt3g.core.G3Int) => 1
"compress_flags_D00B-280_gain" (spt3g.core.G3Int) => 1
"compress_flags_D00B-280_offset" (spt3g.core.G3Double) => 0
"compress_flags_D00B-280_units" (spt3g.core.G3Int) => 1
"compress_flags_D01A-280_gain" (spt3g.core.G3Int) => 1
"compress_flags_D01A-280_offset" (spt3g.core.G3Double) => 0
"compress_flags_D01A-280_units" (spt3g.core.G3Int) => 1
"compress_flags_D01B-280_gain" (spt3g.core.G3Int) => 1
"compress_flags_D01B-280_offset" (spt3g.core.G3Double) => 0
"compress_flags_D01B-280_units" (spt3g.core.G3Int) => 1
"compress_flags_D02A-280_gain" (spt3g.core.G3Int) => 1
"compress_flags_D02A-280_offset" (spt3g.core.G3Double) => 0
"compress_

In [84]:
print(firstscanframe["boresight_radec"][0].__class__.__dict__)
print(firstscanframe["boresight_radec"][0].__class__.__dict__.keys())

quat = firstscanframe["boresight_radec"][0]
print(quat)
print((quat.a ** 2 + quat.b ** 2 + quat.c ** 2 + quat.d ** 2) ** 0.5)

{'__module__': 'spt3g.core', '__doc__': 'Representation of a quaternion. Data in a,b,c,d.', '__reduce__': <Boost.Python.function object at 0x564d9afdf040>, '__instance_size__': 56, '__init__': <Boost.Python.function object at 0x564d9bc048b0>, 'a': <property object at 0x7ffa4ff7a890>, 'b': <property object at 0x7ffa4ff7a8e0>, 'c': <property object at 0x7ffa4ff7a930>, 'd': <property object at 0x7ffa7c53d5d0>, '__invert__': <Boost.Python.function object at 0x564d9bc04bb0>, '__eq__': <Boost.Python.function object at 0x564d9bc04c10>, '__ne__': <Boost.Python.function object at 0x564d9bc04ee0>, '__add__': <Boost.Python.function object at 0x564d9bc04f60>, '__iadd__': <Boost.Python.function object at 0x564d9bc04fe0>, '__sub__': <Boost.Python.function object at 0x564d9bc05060>, '__isub__': <Boost.Python.function object at 0x564d9bc050e0>, '__mul__': <Boost.Python.function object at 0x564d9bc051e0>, '__rmul__': <Boost.Python.function object at 0x564d9bc05260>, '__imul__': <Boost.Python.function o